<h1> Exercise 4.4

In [2]:
library(ivreg)

In [5]:
data <- read.csv("./data/TrainExer44.csv")
names(data)[1]<-< "OBS"

In [6]:
summary(data)

      OBS             GC              PG                 RI       
 Min.   :1970   Min.   :6.634   Min.   :-0.37973   Min.   :2.943  
 1st Qu.:1977   1st Qu.:6.742   1st Qu.:-0.19088   1st Qu.:3.174  
 Median :1984   Median :6.851   Median :-0.12693   Median :3.341  
 Mean   :1984   Mean   :6.921   Mean   :-0.04809   Mean   :3.332  
 3rd Qu.:1992   3rd Qu.:7.054   3rd Qu.: 0.06909   3rd Qu.:3.515  
 Max.   :1999   Max.   :7.301   Max.   : 0.39773   Max.   :3.685  
      RPN                 RPT                RPU          
 Min.   :-0.176815   Min.   :-0.27945   Min.   :-0.29515  
 1st Qu.:-0.085284   1st Qu.:-0.10826   1st Qu.:-0.18228  
 Median : 0.005886   Median : 0.02189   Median :-0.09707  
 Mean   : 0.032685   Mean   :-0.01038   Mean   :-0.10364  
 3rd Qu.: 0.150454   3rd Qu.: 0.08865   3rd Qu.:-0.01417  
 Max.   : 0.311872   Max.   : 0.17116   Max.   : 0.07952  

<h3> Part (a)<h3> 

It might be due to 2 factors:<br><br>
(1) strategic behaviour: gasoline producer might increase their price if they the demand for gasoline is high. As such, the price of gasoline and demand for gasoline is related to a hidden variable (for instance, exceptional event such as gasoline shortage) which is captured in epsilon. Hence, price of gasoline might be correlated to epsilon.<br><br>
(2) omitted variables: price of gasoline might be correlated to excluded explanatory variables such as RPT, RPN and RPU. These excluded variables, if relevant in the true model, are captured in epsilon. Hence, price of gasoline might be correlated to epsilon.

<h3> Part (b) </h3>

In [7]:
OLS1 <- lm(PG ~ RI + RPT + RPN + RPU, data=data)

In [8]:
X_hat <- predict(OLS1)
combined_data <- cbind(data, X_hat) 

In [9]:
OLS2 <- lm(GC ~ X_hat + RI, data=combined_data)
# summary(OLS2)
sprintf("Beta2 = %.3f", coef(OLS2)[2])

[1] "Beta2 = -0.544"

<h3> Part (c) </h3>

In [10]:
mod_data <- data
names(mod_data)[3] <- "X_hat"
y_hat <- predict(OLS2, newdata=mod_data)

In [11]:
e <- mod_data$GC - y_hat

In [12]:
mod_data$e <- e  

In [13]:
reg <- lm(e ~ RI + RPT + RPN + RPU, data=mod_data)
sprintf("R-squared = %.3f", summary(reg)$r.squared)

[1] "R-squared = 0.104"

In [14]:
n <- nrow(mod_data)
k <- 2
m <- 5 #include intercept
nr2 <- n*summary(reg)$r.squared

In [19]:
nr2

[1] 3.124783

In [20]:
pval <- pchisq(nr2, df=(m-k), lower.tail=FALSE)
# qchisq(0.05, df=(m-k), lower.tail=FALSE)
sprintf("p-value = %.3f > 0.05.  Do not reject H0", pval)

[1] "p-value = 0.373 > 0.05.  Do not reject H0"

<h3> Alternative </h3>

Formula format: <br>
(1) X | Z<br>
(2) X_endogeneous | X_exogenous | Z

In [31]:
SLS <- ivreg(formula= GC ~ RI | PG | RPT + RPN + RPU, data=data)

In [32]:
summary(SLS)


Call:
ivreg(formula = GC ~ RI | PG | RPT + RPN + RPU, data = data)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0444538 -0.0140865  0.0009991  0.0193270  0.0411568 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  5.01370    0.08391   59.75   <2e-16 ***
PG          -0.54445    0.02895  -18.81   <2e-16 ***
RI           0.56466    0.02539   22.24   <2e-16 ***

Diagnostic tests:
                 df1 df2 statistic  p-value    
Weak instruments   3  25    43.452 4.58e-10 ***
Wu-Hausman         1  26     2.241    0.146    
Sargan             2  NA     3.125    0.210    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.02403 on 27 degrees of freedom
Multiple R-Squared: 0.987,	Adjusted R-squared: 0.986 
Wald test:  1001 on 2 and 27 DF,  p-value: < 2.2e-16 
